In [ ]:
import csv
import random
import math

In [ ]:
def load_data(filepath= './data/patient_vitals.csv'):
    """Reads patient data from a csv and returns a list of dictionaries.
    Validates numeric inegrity for the heart rate data"""
    patients = []
    try:
        # Open the file using context managers
        with open(filepath, mode='r', encoding='utf-8-sig') as csvfile:
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                try:
                    # Rename columns and type casting
                    patient_record = {
                        'patient_id': int(row['Subject ID']),
                        'age': random.randint(20, 80),
                        'temperature': float(row['Temperature reading 1']),
                        'heart_rate': int(row['Heart rate reading 1']),
                        'systolic_bp': int(row['Systolic blood pressure reading 1']),
                        'oxygen_saturation': int(row['Oxygen saturation reading 1'])
                    }
                    patients.append(patient_record)
                    
                except ValueError as ve:
                    print(f"Skipping row for Subject {row.get('Subject ID')}: {ve}")
                    
                except KeyError as ke:
                    print(f"Missing column error: {ke}")
                    return None
                
        return patients

    except FileNotFoundError:
        print(f"File {filepath} not found.")
        return []
        
data = load_data('./data/patient_vitals.csv')

# display the result
if data:
    print(f"Successfully loaded {len(data)} patients record")
    for patient in data[:5]:
        print(patient)
else:
    print("No valid patient data loaded.")   

In [ ]:

# calculate mean value
def calculate_mean(values):
    """Return the mean of a list of numbers."""
    if not values:
        return 0
    return sum(values) / len(values)

def calculate_median(values):
    """Return the median of a list of numbers."""
    if not values:
        return 0
    sorted_vals = sorted(values)
    n = len(sorted_vals)
    mid = n // 2
    
    if n % 2 == 0:
        return (sorted_vals[mid - 1] + sorted_vals[mid]) / 2
    else:
        return sorted_vals[mid]
    
def calculate_mode(values):
    """Returns the most frequently occurring value(s) in the list."""
    if not values:
        return []
    
    frequency = {}
    for value in values:
        frequency[value] = frequency.get(value, 0) + 1
    
    max_freq = max(frequency.values())
    modes = [key for key, val in frequency.items() if val == max_freq]
    
    return modes

def calculate_outliers(values):
    """Returns a list of outliers using the IQR method."""
    if not values:
        return []
    
    sorted_vals = sorted(values)
    n = len(sorted_vals)
    
    Q1 = calculate_median(sorted_vals[:n//2])
    Q3 = calculate_median(sorted_vals[(n+1)//2:])
    IQR = Q3 - Q1
    
    lower_bound = Q1 - ( 1.5 * IQR)
    upper_bound = Q3 + (1.5 * IQR)
    
    outliers = [value for value in values if value < lower_bound or value > upper_bound]
    
    return outliers

In [ ]:
# load Data
data = load_data('./data/patient_vitals.csv')

if data:
    heart_rates = [patient['heart_rate'] for patient in data if 'heart_rate' in patient]
    
    mean_hr = calculate_mean(heart_rates)
    median_hr = calculate_median(heart_rates)
    mode_hr = calculate_mode(heart_rates)
    outliers_hr = calculate_outliers(heart_rates)

In [ ]:
def test_dev_phase2():
    
    test_list = [1, 2, 3, 4, 5, 6, 100]
    
    mean_test = calculate_mean(test_list)
    outliers_test = calculate_outliers(test_list)
    
    print(f"Test List: {test_list}")
    print(f" Calculated Mean: {mean_test}")
    print(f" Dectected Outliers: {outliers_test}")
    
    if 100 in outliers_test:
        print("Dev Phase 2 Success!: Outlier is correct")
    else:
        print("Dev Phase 2 Failure: Outlier not detected")
        
test_dev_phase2()

In [ ]:
def generate_summary_report(data):
    """Generates a summary report of the patient data."""
    if not data:
        return "No data to summarize."
    
    heart_rates = [patient['heart_rate'] for patient in data if 'heart_rate' in patient]
    temperatures = [patient['temperature'] for patient in data if 'temperature' in patient]
    
    mean_hr = calculate_mean(heart_rates)
    median_hr = calculate_median(heart_rates)
    mode_hr = calculate_mode(heart_rates)
    outliers_hr = calculate_outliers(heart_rates)
    
    # 3. Identify Flagged Patients (New Logic)
    flagged_patients = []
    for p in data:
        if p['heart_rate'] in outliers_hr:
            flagged_patients.append(p['patient_id'])
            
    # Temperature Stats
    outliers_temp = calculate_outliers(temperatures)
    
    report = (
        f"{'='*30}\n"
        f"HEALTH CLINIC SUMMARY REPORT\n"
        f"{'='*30}\n"
        f"Total Patients: {len(data)}\n\n"
        f"Heart Rate Summary Report:\n"
        f"Mean: {mean_hr:.2f}\n"
        f"Median: {median_hr}\n"
        f"Mode: {mode_hr}\n"
        f"Outliers Found: {outliers_hr}\n"
        f"TEMPERATURE STATS:\n"
        f"- Mean: {calculate_mean(temperatures):.2f}°C\n"
        f"- Outliers Found: {outliers_temp}\n\n"
        f"🚨 RISK ASSESSMENT:\n"
        f"The following Patient IDs require immediate review: {flagged_patients if flagged_patients else 'None'}\n"
        f"{'='*30}"
    )
    
    return report

print(generate_summary_report(data))